In [ ]:
import os
import pandas as pd
from pandas.errors import EmptyDataError

# === CONFIGURATION ===
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/1976_SA_1/salyan/model"
DAILY_OUTDIR = os.path.join(BASE_DIR, "SA_output_basin")
CSV_OUTDIR = os.path.join(BASE_DIR, "SA_output_basin_csv")  # New folder for CSVs
os.makedirs(CSV_OUTDIR, exist_ok=True)

# NUM_DEFS_FOLDERS should already be set before running this code

# Variables of interest
VARIABLES = ['day', 'month', 'year', 'streamflow', 'laiTREE', 'evap', 'trans']

# === PROCESSING LOOP ===
for i in range(1, NUM_DEFS_FOLDERS + 1):
    result_file = os.path.join(DAILY_OUTDIR, f"Salyan{i}_basin.daily")
    out_csv = os.path.join(CSV_OUTDIR, f"basin{i}.csv")
    if not os.path.exists(result_file):
        print(f"WARNING: {result_file} not found! Skipping...")
        continue
    # Read the header line
    try:
        with open(result_file, 'r') as f:
            header = f.readline().strip().split()
    except Exception as e:
        print(f"ERROR: Could not read {result_file}: {e}")
        continue
    # Try to read the data into a DataFrame
    try:
        df = pd.read_csv(
            result_file,
            delim_whitespace=True,
            comment="#",
            skiprows=1,
            header=None,
            names=header
        )
    except EmptyDataError:
        print(f"EmptyDataError: {result_file} is empty or malformed; skipped.")
        continue
    except Exception as e:
        print(f"ERROR reading {result_file}: {e}")
        continue
    if df.empty:
        print(f"WARNING: {result_file} contains no data rows. Skipping...")
        continue
    # Ensure required date components exist and are integers
    date_missing = False
    for col in ['year', 'month', 'day']:
        if col in df.columns:
            try:
                df[col] = df[col].astype(int)
            except Exception as e:
                print(f"ERROR: Could not convert column '{col}' in {result_file}: {e}")
                date_missing = True
                break
        else:
            print(f"WARNING: Column '{col}' missing in {result_file}. Skipping...")
            date_missing = True
            break
    if date_missing:
        continue
    # Build date column
    df['date'] = pd.to_datetime(
        df[['year', 'month', 'day']].astype(str).agg('-'.join, axis=1),
        format="%Y-%m-%d",
        errors='coerce'
    )
    df = df.dropna(subset=['date'])
    if df.empty:
        print(f"WARNING: All dates invalid in {result_file}. Skipping...")
        continue
    # Extract needed columns safely
    missing_vars = [v for v in ['streamflow', 'laiTREE', 'evap', 'trans'] if v not in df.columns]
    if missing_vars:
        print(f"WARNING: Missing variables {missing_vars} in {result_file}. Skipping...")
        continue
    outdf = df[['date', 'streamflow', 'laiTREE', 'evap', 'trans']].copy()
    outdf.loc[:, 'date'] = outdf['date'].dt.strftime("%Y-%m-%d")
    outdf.to_csv(out_csv, index=False)
    print(f"{out_csv} written.")
print("All files processed.")
